In [1]:
import tensorflow as tf
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from time import time

from numpy.random import seed
seed(12)
from tensorflow import set_random_seed
set_random_seed(12)

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12)
from scipy import stats
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


from sklearn.metrics import classification_report, f1_score, accuracy_score
import glob
import warnings
import pandas as pd
import numpy as np

import os

Using TensorFlow backend.


## Load dataset

In [2]:
path_dataset = "Dataset/WISDM/WISDM_ar_v1.1/a.txt"

In [3]:
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
df = pd.read_csv(path_dataset, header = None, names = columns )
df

,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164
5,33,Jogging,49106442306000,1.375655,-2.492524,-6.510526
6,33,Jogging,49106542312000,-0.612916,10.569390,5.706926
7,33,Jogging,49106652389000,-0.503953,13.947236,7.055340
8,33,Jogging,49106762313000,-8.430995,11.413852,5.134871
9,33,Jogging,49106872299000,0.953424,1.375655,1.648062


In [4]:
null_counts = df.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
user         0
activity     0
timestamp    0
x-axis       0
y-axis       0
z-axis       1
dtype: int64


In [5]:
df = df.dropna()

In [6]:
N_TIME_STEPS = 200
N_FEATURES = 3
step = 20
segments = []
labels = []
for i in range(0, len(df) - N_TIME_STEPS, step):
    xs = df['x-axis'].values[i: i + N_TIME_STEPS]
    ys = df['y-axis'].values[i: i + N_TIME_STEPS]
    zs = df['z-axis'].values[i: i + N_TIME_STEPS]
    label = stats.mode(df['activity'][i: i + N_TIME_STEPS])[0][0]
    segments.append([xs, ys, zs])
    labels.append(label)

C:\Users\fifamd\Anaconda3\envs\tf\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [7]:
reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)
labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

In [8]:
RANDOM_SEED = 42

In [9]:
X_train, X_test, y_train, y_test = train_test_split(reshaped_segments, labels, test_size=0.2, random_state=RANDOM_SEED)

## Transfer

In [10]:
from keras.models import Model
from keras.layers import Input, Dense,LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from numpy.random import seed
seed(12)
from tensorflow import set_random_seed
set_random_seed(12)

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12)

from sklearn.metrics import classification_report, f1_score, accuracy_score

In [11]:
def f1_score_metric(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 5 * precision * recall / (4 * precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)

In [12]:
print(X_train.shape)
print(y_train.shape)
print(reshaped_segments.shape)
numOfRows = reshaped_segments.shape[1]
numOfColumns = reshaped_segments.shape[2]
filters = 128
Epochs = 20
batchSize = 10
num_class = 6
print(numOfRows , numOfColumns)
print(y_train)

(43920, 200, 3)
(43920, 6)
(54901, 200, 3)
200 3
[[ 0.  0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 ..., 
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.]]


In [1]:
y_train = np.reshape(y_train,(43920,6))
y = np.array(y_train)
print(y.shape)

NameError: name 'np' is not defined

In [14]:
from keras.models import Sequential
from keras.layers import Activation, Dense

In [28]:
def TCNN():
    base_model1 = Input(shape=(numOfRows,numOfColumns))
    base_model = Conv1D(filters,(15),activation = 'relu')(base_model1)
    base_model = MaxPooling1D(pool_size=4)(base_model)
    base_model = Conv1D(filters,(15), activation = 'relu')(base_model)
    base_model = MaxPooling1D(pool_size=5)(base_model)

    base_model = LSTM(128)(base_model)

    base_model = Dense(256)(base_model)
    base_model = Dropout(0.2)(base_model)
    base_model = Dense(6 ,activation="softmax")(base_model)
    model = Model(inputs=[base_model1], outputs=base_model)
    adam = optimizers.Adam(lr = 0.001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [29]:
m = TCNN()
m.summary()

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                                verbose=1, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=20, verbose=0, mode='max')

model_1_path = "./model/model_1/{}.h5".format("model_1")
checkpoint = ModelCheckpoint(model_1_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

history = m.fit(X_train,y_train,epochs = Epochs, callbacks=callbacks_list, verbose =1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 200, 3)            0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 186, 128)          5888      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 46, 128)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 32, 128)           245888    
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 6, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_9 (Dense)              (None, 256)               33024     
__________

C:\Users\fifamd\Anaconda3\envs\tf\lib\site-packages\keras\callbacks.py:406: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
C:\Users\fifamd\Anaconda3\envs\tf\lib\site-packages\keras\callbacks.py:497: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Users\fifamd\Anaconda3\envs\tf\lib\site-packages\keras\callbacks.py:898: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


43920/43920 [==============================] - 86s 2ms/step - loss: 0.1571 - acc: 0.9472: 1s - loss: 
Epoch 3/20
43920/43920 [==============================] - 243s 6ms/step - loss: 0.1134 - acc: 0.9620
Epoch 4/20
43920/43920 [==============================] - 222s 5ms/step - loss: 0.0998 - acc: 0.9662
Epoch 5/20
43920/43920 [==============================] - 219s 5ms/step - loss: 0.0854 - acc: 0.9712
Epoch 6/20
43920/43920 [==============================] - 225s 5ms/step - loss: 0.0768 - acc: 0.9742
Epoch 7/20
43920/43920 [==============================] - 208s 5ms/step - loss: 0.0739 - acc: 0.9748
Epoch 8/20
43920/43920 [==============================] - 230s 5ms/step - loss: 0.0690 - acc: 0.9773
Epoch 9/20
43920/43920 [==============================] - 235s 5ms/step - loss: 0.0640 - acc: 0.9785
Epoch 10/20
43920/43920 [==============================] - 218s 5ms/step - loss: 0.0658 - acc: 0.9774
Epoch 11/20
43920/43920 [==============================] - 216s 5ms/step - loss: 0.0641 -

AttributeError: 'History' object has no attribute 'save_weight'

In [33]:
m.save_weights("./model/model_1.h5")

In [ ]:
def RNN():
    model = Sequential()
    model.add(Conv1D(filters,(2),input_shape=(200,3)))
    model.add(Activation('relu'))
    model.add(LSTM(10, go_backwards = True))
    model.add(Dense(6,activation='softmax'))
    adam = optimizers.Adam(lr = 0.001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model = TCNN()
model.summary()

for layer in model.layers:
    layer.trainable = False
    
base_model1 = Input(shape=(numOfRows,numOfColumns))
x1 = model.output
x = Dense(256)(x1)
x = Dropout(0.2)(x)
x = Dense(128)(x)
x = Dropout(0.2)(x)
output = Dense(num_class, activation='softmax')(x)
print(x1)
model_ = Model(inputs=[base_model1], outputs = output)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                                verbose=1, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=20, verbose=0, mode='max')

model_1_path = "./model/model_1/{}.h5".format("model_1")
checkpoint = ModelCheckpoint(model_1_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

history = model_.fit(X_train,y_train,epochs = Epochs, call_back=callback_list, verbose =1)
history.save_weight("./model/model_1.h5")

# Create and compile model
model_1 = Model(inputs=model.input, outputs=output)
model_1.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.0001),metrics=["accuracy", f1_score_metric])


# Load last epoch model
pretrined_model_1_path = "./model/model_1.h5"
model_1.load_weights(pretrined_model_1_path)

history = model_1.fit(X_train,y_train,epochs = Epochs, call_back=callback_list)
history.save_weight("./model/model_1_complete.h5")

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False
base_model.summary()

# score = history.evaluate(X_test,y_test,verbose=2)
print('Baseline Error: %.2f%%' %(100-score[1]*100))